# Install Dependencies

## Install detectron2: (Colab has CUDA 10.1 + torch 1.7)
See [Detectron2 Installation Guide](https://detectron2.readthedocs.io/tutorials/install.html) for instructions.

In [1]:
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())
!gcc --version

## Setup Detectron2 Logger

In [2]:
import detectron2
# from detectron2.utils.logger import setup_logger
# setup_logger()

## Import Some Common Libraries

In [3]:
import numpy as np
import os, json, random
from google.colab.patches import cv2_imshow

## Import Some Common Detectron2 Utilities

In [4]:
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

## Mount Your Google Drive

In [5]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

After mounting the drive, look in the Colab file explorer (at the left side of the browser by default).

In [6]:
PATH_TO_IMGS = "/content/drive/Shareddrives/Weed_Detection"

## Unzip Files

In [7]:
!unzip "/content/drive/Shareddrives/Weed_Detection/Testing_Data/1_1.zip"
!unzip "/content/drive/Shareddrives/Weed_Detection/1_2.zip"

## Register COCO Instances

In [8]:
from detectron2.data.datasets import register_coco_instances
register_coco_instances("garden_training_set", {}, "/content/drive/Shareddrives/Weed_Detection/1_2.json", "/content/")
register_coco_instances("garden_testing_set", {}, "/content/drive/Shareddrives/Weed_Detection/Testing_Data/1_2.json", "/content/")

## Check That the Data Loaded Correctly with Visualization

In [9]:
import random
from detectron2.utils.visualizer import Visualizer

dataset_dicts = DatasetCatalog.get("garden_training_set")
meta_data = MetadataCatalog.get("garden_training_set")

## Remove Empty Annotations from Dictionary

In [10]:
dataset_dicts

In [11]:
dataset_dicts = DatasetCatalog.get("garden_training_set")
for entry in dataset_dicts:
    print(entry)

## Train the Model

Fine-tune a COCO-pretrained R50-FPN Mask R-CNN model.

In [12]:
from detectron2.engine import DefaultTrainer

cfg = get_cfg()
cfg.DATASETS.TRAIN = ("garden_training_set",)
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.00025
cfg.SOLVER.MAX_ITER = 300
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 2

os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = DefaultTrainer(cfg) 
trainer.resume_or_load(resume=False)
trainer.train()

## Visualize Training Results

In [13]:
from detectron2.utils.visualizer import ColorMode

cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.8
predictor = DefaultPredictor(cfg)

dataset_dicts = DatasetCatalog.get("garden_testing_set")
for d in random.sample(dataset_dicts, 3):
    im = cv2.imread(d["file_name"])
    outputs = predictor(im)
    v = Visualizer(im[:, :, ::-1],
                   metadata=MetadataCatalog.get("garden_testing_set"),
                   scale=0.8,
                   instance_mode=ColorMode.IMAGE)
    v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    cv2_imshow(v.get_image()[:, :, ::-1])